In [168]:
using Oscar
using Combinatorics
pm = Polymake

include("matroid_realization.jl");
include("reduceIdeals.jl")
include("tscCoordRing.jl");


In [21]:
d4n8 = vec(readlines("d4n8.dat"));

In [22]:
#wierd matroid
M270 = matroid_from_revlex_basis_encoding(d4n8[270], 4, 8)

Matroid of rank 4 on 8 elements

In [27]:
AG270  =automorphism_group(M270)

Group([ (1,2)(7,8), (2,3)(6,7), (3,5)(4,6), (1,2)(3,4) ])

In [7]:
nonbases(M270) 

14-element Vector{Vector{Int64}}:
 [1, 2, 3, 4]
 [1, 2, 5, 6]
 [1, 2, 7, 8]
 [1, 3, 5, 7]
 [1, 3, 6, 8]
 [1, 4, 5, 8]
 [1, 4, 6, 7]
 [2, 3, 5, 8]
 [2, 3, 6, 7]
 [2, 4, 5, 7]
 [2, 4, 6, 8]
 [3, 4, 5, 6]
 [3, 4, 7, 8]
 [5, 6, 7, 8]

In [ ]:
#note that the matroid is connected:
is_connected(M270)

In [ ]:
#that is, it has 1 connected component (not a direct sum of smaller matroids)
n_connected_components(M270)

In [8]:
#these are the circuits. Note that we have all rank 3 circuits, 
#such that each element is contained in exactly 7. This is odd, since we (Dan and myself) expected connected
#matroids to have at least one circuit of cardinality rank+1
circuits(M270)

14-element Vector{Vector{Int64}}:
 [1, 2, 3, 4]
 [1, 2, 5, 6]
 [1, 2, 7, 8]
 [1, 3, 5, 7]
 [1, 3, 6, 8]
 [1, 4, 5, 8]
 [1, 4, 6, 7]
 [2, 3, 5, 8]
 [2, 3, 6, 7]
 [2, 4, 5, 7]
 [2, 4, 6, 8]
 [3, 4, 5, 6]
 [3, 4, 7, 8]
 [5, 6, 7, 8]

In [9]:
B= bases(M270)

56-element Vector{Vector{Int64}}:
 [1, 2, 3, 5]
 [1, 2, 4, 5]
 [1, 3, 4, 5]
 [2, 3, 4, 5]
 [1, 2, 3, 6]
 [1, 2, 4, 6]
 [1, 3, 4, 6]
 [2, 3, 4, 6]
 [1, 3, 5, 6]
 [2, 3, 5, 6]
 [1, 4, 5, 6]
 [2, 4, 5, 6]
 [1, 2, 3, 7]
 ⋮
 [1, 3, 7, 8]
 [2, 3, 7, 8]
 [1, 4, 7, 8]
 [2, 4, 7, 8]
 [1, 5, 7, 8]
 [2, 5, 7, 8]
 [3, 5, 7, 8]
 [4, 5, 7, 8]
 [1, 6, 7, 8]
 [2, 6, 7, 8]
 [3, 6, 7, 8]
 [4, 6, 7, 8]

In [10]:
#automorphism group. I was having trouble getting the code from git, so I wrote my own for this special case

edges = []
for i in 1:8
    bi = [[i,j+8] for j in 1:length(B) if i in B[j]]
    edges = union(edges,bi)
end

eedges = Vector{Vector{Int64}}(edges)
G = pm.graph.graph_from_edges(eedges)   
GG = Graph{Undirected}(G.ADJACENCY)

describe(automorphism_group(GG))

"(C2 x C2 x C2) : PSL(3,2)"

In [156]:
function new_TSC(M, F)
    d = rank(M)
    n = length(matroid_groundset(M))
    Bs = bases(M)
    NBs = nonbases(M)
    B = Bs[1]
    R,x = makePolyRing(d,n,F)
   
    Mx = reducedCoordinateMatrix(M,d,n,B,R,x)
    I = ideal(unique!([det(Mx[:, nb]) for nb in NBs ]))
    toInvert = unique!([det(Mx[:, b])  for b in Bs])
    
    return (I,toInvert)
end

new_TSC (generic function with 3 methods)

We can interpret the output above using this key: https://docs.oscar-system.org/dev/Groups/basics/#describe-Tuple{Oscar.GAPGroup}

That is, the automorphism group of the weird matroid is the semi direct product of $\mathbb{Z}/2\mathbb{Z}$ with $PSL(3,2)$, where $PSL(3,2)$  denotes the projective special linear group of $3\times 3$ matrices with entries from $F_2$

In [149]:
function TSC_to_reduced_expression(M, F)
    
    charts = bases(M)
    #println("1")
    A = charts[1]
   # println("2")
    RQ = new_TSC(M,F)
    #println("3")
    R = parent(RQ[1][1])
    #println("4")
    Sgens = [s for s in RQ[2]]#new 13.1.2023
    #print(length(Sgens),"\n")
    #print(length(gens(RQ[1])))
    #println("5")
    I = reduce_ideal_full(gens(RQ[1]), Sgens, R, gens(R), false)
   # println("6")
    
     varlist = 
    if I isa String
        
        return I
        
     #I[1] = ideal generators, I[2] = subgroup generators   
    else
        
       Iclean = unique!([clean(f,R) for f in I[1]])
    
            
       return (Iclean, I[2])
        
       # return(I[1],I[2])
        
    end
end

TSC_to_reduced_expression (generic function with 2 methods)

In [155]:
TSC_to_reduced_expression(M270, ZZ)[1]

2-element Vector{MPolyDecRingElem{fmpz, fmpz_mpoly}}:
 0
 2

In [164]:
M = matroid_from_revlex_basis_encoding(d4n8[64], 4, 8)

Matroid of rank 4 on 8 elements

In [165]:
TSC_to_reduced_expression(M, QQ)

(MPolyDecRingElem{fmpq, fmpq_mpoly}[0], MPolyDecRingElem{fmpq, fmpq_mpoly}[1, x_{3, 1}, -x_{3, 1}*x_{2, 4}, x_{3, 1}*x_{1, 2}*x_{3, 3}^2*x_{4, 4}^2 - x_{3, 1}*x_{1, 2}*x_{3, 3}*x_{4, 3}*x_{3, 4}*x_{4, 4} - x_{3, 1}*x_{4, 2}*x_{3, 3}^2*x_{1, 4}*x_{4, 4} + x_{3, 1}*x_{4, 2}*x_{3, 3}*x_{4, 3}*x_{1, 4}*x_{3, 4}, x_{4, 2}, x_{3, 1}*x_{4, 2}, -x_{3, 1}*x_{4, 2}*x_{2, 4}, x_{4, 2}*x_{3, 3}*x_{2, 4}, -x_{1, 2}, x_{3, 1}*x_{4, 2}*x_{3, 3}*x_{2, 4}  …  x_{4, 3}*x_{2, 4}, x_{1, 2}*x_{3, 3}^2*x_{4, 4}^3 - x_{1, 2}*x_{3, 3}*x_{4, 3}*x_{3, 4}*x_{4, 4}^2 + x_{1, 2}*x_{4, 3}^2*x_{3, 4}^2*x_{4, 4} - x_{4, 2}*x_{3, 3}^2*x_{1, 4}*x_{4, 4}^2 + x_{4, 2}*x_{3, 3}*x_{4, 3}*x_{1, 4}*x_{3, 4}*x_{4, 4} - x_{4, 2}*x_{4, 3}^2*x_{1, 4}*x_{3, 4}^2, x_{3, 1}*x_{3, 3}*x_{2, 4}*x_{4, 4}, -x_{3, 1}*x_{1, 2}*x_{3, 3}^3*x_{2, 4}*x_{4, 4}^3 + x_{3, 1}*x_{4, 2}*x_{3, 3}^3*x_{1, 4}*x_{2, 4}*x_{4, 4}^2, -x_{3, 3}*x_{2, 4}, -x_{1, 2}*x_{3, 3}^2*x_{4, 4}^2 + x_{1, 2}*x_{3, 3}*x_{4, 3}*x_{3, 4}*x_{4, 4} - x_{1, 2}*x_{4, 3}^2*x_